In [3]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import time
import random
from numpy.linalg import norm
import matplotlib.lines as mlines

In [4]:
'''
Read, plot, resize data
'''

def plot(img, title = " "):
    
    fig, ax = plt.subplots(figsize=(16,8),dpi = 80)
    ax.imshow(img) 
    fig.suptitle(title)
    plt.show() 
    return

def read_image(path, flag = 1):
    
    image = cv.imread(path, flag)
    
    if flag == 1:
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    return image
 
def resize(img, scale = 20):
    
    width = int(img.shape[1] * scale / 100)
    height = int(img.shape[0] * scale / 100)
    dim = (width, height)
  
    return cv.resize(img, dim, interpolation = cv.INTER_AREA)


In [5]:
def get_patches(image, kernal_dim = 32, stride = 16):
    
    '''
    returns patches over the image with the given specifications
    '''
    
    x = np.arange((kernal_dim - 1)//2, image.shape[1] - (kernal_dim - 1)//2 - 1, stride)
    y = np.arange((kernal_dim - 1)//2, image.shape[0] - (kernal_dim - 1)//2 - 1, stride)
    x,y = np.meshgrid(x,y)
    x = x.flatten() ; y = y.flatten()
    #patch centers
    
    numPatches = len(x) ; patches = np.asarray([])
    for i in range(numPatches):
        if i == 0:
            patches = image[y[i] - (kernal_dim-1)//2:y[i] + (kernal_dim)//2 + 1, x[i] - (kernal_dim-1)//2:x[i] + (kernal_dim)//2 + 1].flatten()
        if i != 0:
            patches = np.vstack((patches, image[y[i] - (kernal_dim-1)//2:y[i] + (kernal_dim)//2 + 1, x[i] - (kernal_dim-1)//2:x[i] + (kernal_dim)//2 + 1].flatten()))
  
    return x, y, patches

#Why use for loops when NumPy exists :)


def get_correlation(image1, image2, kernal_dim = 32, stride = 16):
    
    x1, y1, patchesImage1 = get_patches(image1, kernal_dim, stride)
    x2, y2, patchesImage2 = get_patches(image2, kernal_dim, stride)
    
    
    patchesImage1 = patchesImage1-np.mean(patchesImage1,axis=1).reshape(-1,1) ; patchesImage1 = patchesImage1/(norm(patchesImage1,axis=1).reshape(-1,1)+0.1)
    patchesImage2 = patchesImage2-np.mean(patchesImage2,axis=1).reshape(-1,1) ; patchesImage2 = patchesImage2/(norm(patchesImage2,axis=1).reshape(-1,1)+0.1)
    arr = patchesImage1 @ patchesImage2.T    
    coordinate = np.argmax(arr,axis=1)
    
    return x1, y1, x2[coordinate], y2[coordinate]


def plot_best_match(image1, image2, stereoPairNum, kernal_dim = 32, stride = 16):
    
    x1, y1, x2, y2 = get_correlation(image1, image2, kernal_dim = kernal_dim, stride = stride)
    fig,ax = plt.subplots(dpi=80)
    ax.imshow(np.hstack((image1, image2)))
    l = image1.shape[1]
    
    for i in range(len(y1)):
        ax.plot(np.asarray([x1[i],x2[i]+l]),np.asarray([y1[i],y2[i]]), '--o', linewidth = 1, markersize = 3)
    ax.axis('off')
    fig.suptitle(f"Brute Force Matching among windows in stereo pair {stereoPairNum}", fontsize = 15)

    plt.show()    

In [ ]:
def get_homo(arr):
    
    return np.hstack((arr, np.ones((arr.shape[0], 1))))



def de_homo(arr):
    
    with np.errstate(divide='ignore', invalid='ignore'):
        arr = np.divide(arr,(arr[:,-1].reshape(-1,1)))
    arr[np.isnan(arr)] = 1e9

    return arr[:,:-1]



def newline(l,s):
    a,b,_ = s
    if l[1]==0:
#         return -l[2]/l[1],0,-l[2]/l[1],b
        return -l[2]/l[0],0,  -l[2]/l[0],a
    else:
#         return 0,-l[2]/l[0],a,-l[2]/l[0]-l[1]*a/l[0]
        return 0,-l[2]/l[1],  b,-l[2]/l[1]-l[0]*b/l[1]


def epilines(img1,img2):

    x1,x2,F = get_points_descriptors(img1,img2, display = True)

    
    l2 = get_homo(x1)@F
    l1 = (F@get_homo(x2).T).T
    
    
    fig = plt.figure(figsize=(10,6))
    ax=fig.add_subplot(1,2,1)
    for i in range(len(l1)):
        a,b,c,d = newline(l2[i],img1.shape)
        ax.plot([a,c],[b,d])
        ax.scatter(x1[i,0],x1[i,1])

    print(len(x1))
#     ax.scatter(x1[:,0],x1[:,1])
    ax.imshow(img1)
    ax.axis('off')

    ax=fig.add_subplot(1,2,2)
    for i in range(len(l2)):
        a,b,c,d = newline(l1[i],img2.shape)
        ax.plot([a,c],[b,d])
        ax.scatter(x2[i,0],x2[i,1])

#     ax.scatter(x2[:,0],x2[:,1])
    ax.imshow(img2)
    ax.axis('off')

#     plt.suptitle('Point correspondances and Epilines', fontsize=15)
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0,rect=[0, 0, 1, 0.95])
    plt.show()

    return 

In [ ]:
'''
Utility functions for SIFT keypoints
'''

def SIFT_descriptors(image):
    
    sift = cv.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return keypoints, descriptors


def draw_and_display(image, keypoints):
    
    draw_features = np.zeros(image.shape)
    draw_features = cv.drawKeypoints(image, keypoints, draw_features)
    return draw_features


def SIFT_matching(descriptorL, descriptorR, img1, kp1, img2, kp2, display = False, dense = False):
    
    matcher = cv.BFMatcher(cv.NORM_L2, crossCheck = True)
    #matches = matcher.knnMatch(descriptorL, descriptorR, k = 111)
    matches = matcher.match(descriptorL, descriptorR)
    if len(matches) < 8:
        print("Not enough correspondances!")
        exit(0)
    numPoints = min(max(8, int(0.1 * len(matches))), 100) 
    if dense:
        numPoints = int(0.4 * len(matches))
    matches = sorted(matches, key = lambda x:x.distance)[0:numPoints]
    
    ''''
    Sorting wrt the distance object of each match.
    The closer the matches are, the less erroneous they are likely to be 
    '''
    
    if display:
        
        print(f"The number of matches: {len(matches)}")
        print(f"\033[35m The number of corresponding descriptors: \033[35m {len(matches)}")
        fig, ax = plt.subplots(figsize = (16, 4))
        img1 = cv.drawMatches(img1, kp1, img2, kp2, matches, None, flags = 2|4)
        fig.suptitle(f'The {numPoints} closest correspondences', fontsize = 16)
        ax.imshow(img1)
    
    pts1 = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,2)
    pts2 = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,2)
    return pts1, pts2
   
    
def get_points_descriptors(img1, img2, display = False, dense = False):

    keypointsTrain, descriptorsTrain= SIFT_descriptors(img1) 
    keypointsQuery, descriptorsQuery = SIFT_descriptors(img2)

    x1, x2 = SIFT_matching(descriptorsTrain, descriptorsQuery, img1, keypointsTrain, img2, keypointsQuery, display, dense)
    F, mask = cv.findFundamentalMat(x1,x2)
    x1 = x1[mask.ravel()==1]
    x2 = x2[mask.ravel()==1]

    return x1, x2, F

def get_points_descriptors_flann(img1, img2, display = False, match_th = 0.8):
    
    sift = cv.SIFT_create()
    h,w,c = img1.shape
    kp=[]
    for i in range(1,h,10):
        for j in range(1,w,10):
            kp.append(cv.KeyPoint(i, j, 3))
    
    gray_im1 = cv.cvtColor(img1,cv.COLOR_RGB2GRAY)
    kp1,des1 = sift.compute(gray_im1,kp)

    gray_im2 = cv.cvtColor(img2,cv.COLOR_RGB2GRAY)
    kp2,des2 = sift.compute(gray_im2,kp)
    
    index_params = dict(algorithm = 1, trees = 5)
    search_params = dict(checks=50)
    flann = cv.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    pts1 = []
    pts2 = []
    for i,(m,n) in enumerate(matches):
        if m.distance < match_th*n.distance:
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
    pts1 = np.float32(pts1)
    pts2 = np.float32(pts2)
    
    
    F, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_RANSAC)
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]

    return pts1,pts2,F

In [ ]:
"""
Stereo rectification 
"""

def stereo_rectification(img1,img2, display = False):
    
    pts1, pts2, F = get_points_descriptors_flann(img1,img2, display = True)
    
    p, H1, H2 = cv.stereoRectifyUncalibrated(pts1, pts2, F, img2.shape[0:2][::-1])
        
    H3 = H1.dot(H2)
    rect_img1 = cv.warpPerspective(img1, H1, img1.shape[0:2][::-1])
    rect_img2 = cv.warpPerspective(img2, H3, img2.shape[0:2][::-1])
    
    
    if display:
        
        plot(rect_img1, title = 'Stero-rectification on image 1')
        plot(rect_img2, title = 'Stero-rectification on image 2')
        

    return rect_img1, rect_img2

In [ ]:
"""
Greedy matching ->  using the epipolar planarity constraint (xTFx' = 0)
"""

def greedy_match(a,b,l=32,s=16):

    x1, y1, lis1 = get_patches(a)
    x2, y2, lis2 = get_patches(b)
    lis1 = lis1-np.mean(lis1,axis=1).reshape(-1,1) ; lis1 = lis1/(norm(lis1,axis=1).reshape(-1,1)+0.1)
    lis2 = lis2-np.mean(lis2,axis=1).reshape(-1,1) ; lis2 = lis2/(norm(lis2,axis=1).reshape(-1,1)+0.1)
    
    
    c1 = np.hstack((x1.reshape(-1,1),y1.reshape(-1,1)))
    c2 = np.hstack((x2.reshape(-1,1),y2.reshape(-1,1)))
    ans = c1*0 

    _,_,F = get_points_descriptors_flann(a,b)   
    l2 = get_homo(c1)@F    ; l2 = l2/(norm(l2[:,:2],axis=1).reshape(-1,1))
    
    for i in range(len(x1)):
        t1 = abs(2*get_homo(c2)@l2[i].T)<=l
        i1 = c2[t1] ; lis21 = lis2[t1]
        if len(i1)!=0:
            ans[i] = i1[np.argmax(lis1[i]@lis21.T)]
        else:
            ans[i]=-1e9
        
    return x1,y1,ans[:,0],ans[:,1]
    

def greedy_match_visualized(a,b,l=32,s=16):
    x1,y1,x2,y2 = greedy_match(a,b,l=l,s=s)
    fig,ax = plt.subplots(dpi=120)
    ax.imshow(np.hstack((a,b)))
    l = a.shape[1]
    for i in range(len(y1)):
        if x2[i]!=-1e9:
            ax.plot(np.asarray([x1[i],x2[i]+l]),np.asarray([y1[i],y2[i]]),'--o')
        
    ax.axis('off')
#    ax.set_title("Greedy Matching using epipolar constraint", fontsize=15)
    plt.show()
    

In [ ]:
# """
# Greedy matching and visualization - using the epipolar constraints
# """

# def greedy_match(a,b,l=32,s=16):
#     x1,y1 = PP(a,l,s) ; lis1 = patches(a,x1,y1,l)
#     x2,y2 = PP(b,l,s) ; lis2 = patches(b,x2,y2,l)
#     lis1 = lis1-np.mean(lis1,axis=1).reshape(-1,1) ; lis1 = lis1/(norm(lis1,axis=1).reshape(-1,1)+0.1)
#     lis2 = lis2-np.mean(lis2,axis=1).reshape(-1,1) ; lis2 = lis2/(norm(lis2,axis=1).reshape(-1,1)+0.1)
    
    
#     c1 = np.hstack((x1.reshape(-1,1),y1.reshape(-1,1)))
#     c2 = np.hstack((x2.reshape(-1,1),y2.reshape(-1,1)))
#     ans = c1*0 

#     _,_,F = sift_matching_RANSAC(a,b)   
#     l2 = homo(c1)@F    ; l2 = l2/(norm(l2[:,:2],axis=1).reshape(-1,1))
    
#     for i in range(len(x1)):
#         t1 = abs(2*homo(c2)@l2[i].T)<=l
#         i1 = c2[t1] ; lis21 = lis2[t1]
#         if len(i1)!=0:
#             ans[i] = i1[np.argmax(lis1[i]@lis21.T)]
#         else:
#             ans[i]=-1e9
        
#     return x1,y1,ans[:,0],ans[:,1]
    

# def greedy_match_visualized(a,b,l=32,s=16):
#     x1,y1,x2,y2 = greedy_match(a,b,l=l,s=s)
#     fig,ax = plt.subplots(dpi=120)
#     ax.imshow(np.hstack((a,b)))
#     l = a.shape[1]
#     for i in range(len(y1)):
#         if x2[i]!=-1e9:
#             ax.plot(np.asarray([x1[i],x2[i]+l]),np.asarray([y1[i],y2[i]]),'--o')
        
#     ax.axis('off')
# #    ax.set_title("Greedy Matching using epipolar constraint", fontsize=15)
#     plt.show()
    
    
# """
# Stereo rectification of images
# """

# def stereo_rectification(img1,img2,show=0):
    
#     pts1, pts2, F = sift_matching_RANSAC (img1,img2)
#     p,H1,H2=cv.stereoRectifyUncalibrated(pts1, pts2, F, img2.shape[0:2][::-1])
        
#     H3 = H1.dot(H2)
#     rect_img1 = cv.warpPerspective(img1, H1, img1.shape[0:2][::-1])
#     rect_img2 = cv.warpPerspective(img2, H3, img2.shape[0:2][::-1])
    
#     if show==1:
#         plot_figure([rect_img1, rect_img2],1,2,text = 'Stero-rectification')

#     return rect_img1, rect_img2


# def newline(l,s):
#     a,b,_ = s
#     if l[1]==0:
# #         return -l[2]/l[1],0,-l[2]/l[1],b
#         return -l[2]/l[0],0,  -l[2]/l[0],a
#     else:
# #         return 0,-l[2]/l[0],a,-l[2]/l[0]-l[1]*a/l[0]
#         return 0,-l[2]/l[1],  b,-l[2]/l[1]-l[0]*b/l[1]


# def epilines(img1,img2):

#     x1,x2,F = sift_matching_RANSAC(img1,img2)
    
# #     d = np.isclose(0,np.diag(homo(x1)@F@homo(x2).T),atol=1)
# #     x1 = x1[d] ; x2 = x2[d]

#     l2 = homo(x1)@F
#     l1 = (F@homo(x2).T).T
    
#     print(F)
    
#     fig = plt.figure(figsize=(10,6))
#     ax=fig.add_subplot(1,2,1)
#     for i in range(len(l1)):
#         a,b,c,d = newline(l2[i],img1.shape)
#         ax.plot([a,c],[b,d])
#         ax.scatter(x1[i,0],x1[i,1])

#     print(len(x1))
# #     ax.scatter(x1[:,0],x1[:,1])
#     ax.imshow(img1)
#     ax.axis('off')

#     ax=fig.add_subplot(1,2,2)
#     for i in range(len(l2)):
#         a,b,c,d = newline(l1[i],img2.shape)
#         ax.plot([a,c],[b,d])
#         ax.scatter(x2[i,0],x2[i,1])

# #     ax.scatter(x2[:,0],x2[:,1])
#     ax.imshow(img2)
#     ax.axis('off')

# #     plt.suptitle('Point correspondances and Epilines', fontsize=15)
#     plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0,rect=[0, 0, 1, 0.95])
#     plt.show()

#     return 

# def sift_matching_RANSAC(img1,img2,min_match_cnt=500,match_th =0.8):

#     sift = cv.SIFT_create()
#     h,w,c = img1.shape
#     kp=[]
#     for i in range(1,h,10):
#         for j in range(1,w,10):
#             kp.append(cv.KeyPoint(i, j, 3))
    
#     gray_im1 = cv.cvtColor(img1,cv.COLOR_RGB2GRAY)
#     kp1,des1 = sift.compute(gray_im1,kp)

#     gray_im2 = cv.cvtColor(img2,cv.COLOR_RGB2GRAY)
#     kp2,des2 = sift.compute(gray_im2,kp)
    
#     index_params = dict(algorithm = 1, trees = 5)
#     search_params = dict(checks=50)
#     flann = cv.FlannBasedMatcher(index_params,search_params)
#     matches = flann.knnMatch(des1,des2,k=2)
#     pts1 = []
#     pts2 = []
#     for i,(m,n) in enumerate(matches):
#         if m.distance < match_th*n.distance:
#             pts2.append(kp2[m.trainIdx].pt)
#             pts1.append(kp1[m.queryIdx].pt)
#     pts1 = np.float32(pts1)
#     pts2 = np.float32(pts2)
    
    
    
# #     F,_ = RANSAC_F(pts1,pts2)
    
#     F, mask = cv.findFundamentalMat(pts1,pts2,cv.FM_RANSAC)
#     pts1 = pts1[mask.ravel()==1]
#     pts2 = pts2[mask.ravel()==1]

#     return pts1,pts2,F

# def homo(arr):
#     return np.hstack((arr, np.ones((arr.shape[0], 1))))

# def de_homo(arr):
#     with np.errstate(divide='ignore', invalid='ignore'):
#         arr = np.divide(arr,(arr[:,-1].reshape(-1,1)))
#     arr[np.isnan(arr)] = 1e9

#     return arr[:,:-1]



# def Detect_SIFT(img):
    
#     gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
#     sift = cv.SIFT_create()
#     kp, des = sift.detectAndCompute(gray,None)

#     return kp, des


# def Draw_SIFT(img):
    
#     kp,des = Detect_SIFT(img)
#     img_kp = cv.drawKeypoints(img,kp,None)

#     return img_kp


# def matching(img1, img2):

#     sift = cv.SIFT_create()
#     kp1, des1 = Detect_SIFT(img1)
#     kp2, des2 = Detect_SIFT(img2)

#     bf = cv.BFMatcher(cv.NORM_L2, crossCheck=True)
#     matches = bf.match(des1,des2)
#     matches = sorted(matches, key = lambda x:x.distance)

#     img3 = cv.drawMatches(img1, kp1, img2, kp2, matches[:100], None, flags = 2)

#     x1 = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,2)
#     x2 = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,2)

#     return img3, x1, x2

# # PP is patch points - centre of the patches taken
# def PP(a,l=32,s=16):
#     x = np.arange((l-1)//2,a.shape[1]-(l-1)//2-1,s)
#     y = np.arange((l-1)//2,a.shape[0]-(l-1)//2-1,s)
#     x,y = np.meshgrid(x,y)
#     return x.flatten(),y.flatten()
    

# # returns patches of image
# def patches(a,x,y,l=32):
#     g = len(x) ; lis = np.asarray([])
#     for i in range(g):
#         if i==0:
#             lis = a[y[i]-(l-1)//2:y[i]+(l)//2+1,x[i]-(l-1)//2:x[i]+(l)//2+1].flatten()
#         if i!=0:
#             lis = np.vstack((lis,a[y[i]-(l-1)//2:y[i]+(l)//2+1,x[i]-(l-1)//2:x[i]+(l)//2+1].flatten()))
#     return lis


# # returns best match by Brute force search
# def NCC_best_patch(a,b,l=32,s=16):
    
#     x1,y1 = PP(a,l,s) ; lis1 = patches(a,x1,y1,l)
#     x2,y2 = PP(b,l,s) ; lis2 = patches(b,x2,y2,l)    
    
#     lis1 = lis1-np.mean(lis1,axis=1).reshape(-1,1) ; lis1 = lis1/(norm(lis1,axis=1).reshape(-1,1)+0.1)
#     lis2 = lis2-np.mean(lis2,axis=1).reshape(-1,1) ; lis2 = lis2/(norm(lis2,axis=1).reshape(-1,1)+0.1)
#     arr = lis1@lis2.T    
#     ans = np.argmax(arr,axis=1)
#     return x1,y1,x2[ans],y2[ans]

    
# # best_match_visualized is used to join the point correspondances and plot it on the image
# def best_match_visualized(a,b,l=32,s=16):
#     x1,y1,x2,y2 = NCC_best_patch(a,b,l=l,s=s)
#     fig,ax = plt.subplots(dpi=120)
#     ax.imshow(np.hstack((a,b)))
#     l = a.shape[1]
#     for i in range(len(y1)):
#         ax.plot(np.asarray([x1[i],x2[i]+l]),np.asarray([y1[i],y2[i]]),'--o')
#     ax.axis('off')
# #     ax.set_title("Brute force Matching", fontsize=15)
#     plt.show()    
    
# def plot_figure(img_arr,a,b,text='',c=3):
#     fig = plt.figure(figsize=(b*5,a*5+1))
#     for i in range(1,a+1):
#         for j in range(1,b+1):
#             ax=fig.add_subplot(a,b,(i-1)*b+j)
#             if c==3:
#                 ax.imshow(img_arr[(i-1)*b+j-1])
#             if c==1:
#                 ax.imshow(img_arr[(i-1)*b+j-1],cmap='gray')
#             ax.axis("off")
# #             ax.set_title("image - "+str((i-1)*b+j)+" size "+str(img_arr[(i-1)*b+j-1].shape), fontsize=12)
    
# #     plt.suptitle(text, fontsize=15)
#     #plt.tight_layout()
#     plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0,rect=[0, 0, 1, 0.95])
#     #fig.tight_layout(rect=[0, 0.01, 1, 0.95])
#     plt.show()

#     return 